In [1]:
from pennylane import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
def powerFaultData():

    powerData = pd.read_csv('Testdata.csv')
    
    X = []
    Y = []

    for i in powerData.iterrows():
        X.append([i[1][p] for p in range(0, len(i[1]))])
        Y.append(int(i[1][6]))

    Y = np.asarray(Y).astype(int)

    xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.95, shuffle=True, random_state=42)
    xTrain = preprocessing.normalize(xTrain)
    xTest = preprocessing.normalize(xTest)

    return xTrain, yTrain

In [3]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [4]:
dev = qml.device("default.qubit", wires=6, shots=None)
wires = dev.wires.tolist()

def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        #qml.RZ(x[i], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])
    
def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))
        
adjoint_ansatz = qml.adjoint(ansatz)

def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [5]:
X, Y = powerFaultData()

print("Data: ", X[0])
print("Label: ", Y[0])

Data:  [-8.12146716e-01  4.72210474e-01  3.42651867e-01 -1.82818371e-04
  8.01413195e-05  1.02677050e-04  4.96261294e-03]
Label:  4


/var/folders/g7/wkw4078n4tgdts1d__tj9d140000gn/T/ipykernel_26280/3204019547.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X.append([i[1][p] for p in range(0, len(i[1]))])
/var/folders/g7/wkw4078n4tgdts1d__tj9d140000gn/T/ipykernel_26280/3204019547.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y.append(int(i[1][6]))


In [6]:
init_params = random_params(num_wires=6, num_layers=6)
print(init_params)
kernel_value = kernel(X[0], X[1], init_params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")
print('---------------------------------------------------------------------------------------------')
print('Quantum Circuit: ')
drawer = qml.draw(kernel_circuit)
print(drawer(X[0], X[1], init_params))
print('---------------------------------------------------------------------------------------------')

[[[0.90142964 2.66577638 3.92436245 3.15884105 0.57721573 2.59370191]
  [4.07543409 4.5619082  3.67052112 6.09256244 3.35558994 4.56671337]]

 [[3.67666633 4.9808628  2.97724837 1.24907312 3.01402663 5.01885012]
  [0.82165617 2.66345742 0.19359761 5.25006669 0.54986329 1.82561006]]

 [[0.61148803 5.51822612 5.17279671 3.57906039 1.41783324 0.30922018]
  [2.91190093 5.8166666  5.1581232  1.08884147 2.13048146 0.59999305]]

 [[6.03039782 1.94404575 3.06652189 0.41009607 5.65879874 1.4224221 ]
  [2.84947017 3.19236077 4.91949338 2.94204574 1.86769238 5.54575933]]

 [[5.43262196 5.6896394  0.58684059 3.35157149 3.54191249 3.63275417]
  [6.26005104 5.01390627 2.80532759 4.98793123 6.01394324 1.3981261 ]]

 [[5.89717101 5.72070698 4.11649866 4.4590288  5.58845705 0.11298299]
  [5.87687795 3.24991112 1.70600485 5.78326146 3.25801952 2.94588757]]]
The kernel value between the first and second datapoint is 0.197
-----------------------------------------------------------------------------------

In [7]:
print("Kernel Alignment with Gradient Descent")

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    
    subset = np.random.choice(list(range(len(X))), 4)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")


trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Kernel Alignment with Gradient Descent
Step 10 - Alignment = 0.722
Step 20 - Alignment = 0.746
Step 30 - Alignment = 0.769
Step 40 - Alignment = 0.795
Step 50 - Alignment = 0.830
Step 60 - Alignment = 0.850
Step 70 - Alignment = 0.861
Step 80 - Alignment = 0.873
Step 90 - Alignment = 0.880
Step 100 - Alignment = 0.885
Step 110 - Alignment = 0.888
Step 120 - Alignment = 0.893
Step 130 - Alignment = 0.894
Step 140 - Alignment = 0.896
Step 150 - Alignment = 0.898
Step 160 - Alignment = 0.899
Step 170 - Alignment = 0.901
Step 180 - Alignment = 0.903
Step 190 - Alignment = 0.904
Step 200 - Alignment = 0.904
Step 210 - Alignment = 0.905
Step 220 - Alignment = 0.906
Step 230 - Alignment = 0.906
Step 240 - Alignment = 0.907
Step 250 - Alignment = 0.908
Step 260 - Alignment = 0.908
Step 270 - Alignment = 0.907
Step 280 - Alignment = 0.908
Step 290 - Alignment = 0.908
Step 300 - Alignment = 0.909
Step 310 - Alignment = 0.909
Step 320 - Alignment = 0.909
Step 330 - Alignment = 0.910
Step 340 - Al

In [8]:
def uncertinitySamplingSubset(X, svm_trained, subSize):
	
	probabilities = svm_trained.predict_proba(X)
	entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
	selected_indices = np.argsort(entropy)[:subSize]

	return selected_indices

In [13]:
print("Kernel Alignment with Gradient Descent with Active Learning")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix, probability=True)
svm_trained.fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(500):
    # Choose subset of datapoints to compute the KTA on.
    
    #subset = np.random.choice(list(range(len(X))), 4) 
    trained_kernel = lambda x1, x2: kernel(x1, x2, params)
    trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
    svm_trained = SVC(kernel=trained_kernel_matrix, probability=True).fit(X, Y)
    
    subset = uncertinitySamplingSubset(X, svm_trained=svm_trained, subSize=4)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Kernel Alignment with Gradient Descent with Active Learning
The accuracy of a kernel with trained parameters is 0.833
Step 10 - Alignment = 0.724
Step 20 - Alignment = 0.744
Step 30 - Alignment = 0.777
Step 40 - Alignment = 0.821
Step 50 - Alignment = 0.854
Step 60 - Alignment = 0.872
Step 70 - Alignment = 0.882
Step 80 - Alignment = 0.888
Step 90 - Alignment = 0.892
Step 100 - Alignment = 0.895
Step 110 - Alignment = 0.898
Step 120 - Alignment = 0.900
Step 130 - Alignment = 0.902
Step 140 - Alignment = 0.903
Step 150 - Alignment = 0.904
Step 160 - Alignment = 0.906
Step 170 - Alignment = 0.907
Step 180 - Alignment = 0.908
Step 190 - Alignment = 0.909
Step 200 - Alignment = 0.910
Step 210 - Alignment = 0.910
Step 220 - Alignment = 0.911
Step 230 - Alignment = 0.912
Step 240 - Alignment = 0.913
Step 250 - Alignment = 0.914
Step 260 - Alignment = 0.915
Step 270 - Alignment = 0.916
Step 280 - Alignment = 0.917
Step 290 - Alignment = 0.918
Step 300 - Alignment = 0.919
Step 310 - Alignment 